In [1]:
#pip install mediapipe opencv-python

Se importan las librerias

In [16]:
import cv2
import mediapipe as mp
import numpy as np

procedemos a configurar mediapipe

In [17]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

## Definiciones

Se define una función de cálculo de ángulos

In [18]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

## Programa

In [29]:
cap = cv2.VideoCapture(0)

# definimos variables 
counter = 0 
stage_leg = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            
            # Get coordinates angle leg left            
            shoulder_l = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            knee_l = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle_l = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            
            # Calculate angle leg-shoulder left
            angle_l = calculate_angle(shoulder_l, hip_l, knee_l)
            
            # Visualize angle leg-shoulder left
            cv2.putText(image, str(angle_l.astype(int)), 
                           tuple(np.multiply(hip_l, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # Get coordinates angle leg right 
            shoulder_r = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            hip_r = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            knee_r = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            
            # Calculate angle leg-shoulder right
            angle_r = calculate_angle(shoulder_r, hip_r, knee_r)
            
            # Visualize angle leg-shoulder right
            cv2.putText(image, str(angle_r.astype(int)), 
                           tuple(np.multiply(hip_r, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )            
            
            # Get coordinates angle leg left            
            hip_l = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee_l = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle_l = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            
            # Calculate angle leg left
            angle_l2 = calculate_angle(hip_l, knee_l, ankle_l)
            
            # Visualize angle leg left
            cv2.putText(image, str(angle_l2.astype(int)), 
                           tuple(np.multiply(knee_l, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # Get coordinates angle leg right
            hip_r = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            knee_r = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            ankle_r = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]           
            
            # Calculate angle leg right
            angle_r2 = calculate_angle(hip_r, knee_r, ankle_r)
            
            # Visualize angle leg right
            cv2.putText(image, str(angle_r2.astype(int)), 
                           tuple(np.multiply(knee_r, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
                        
# Definimos las condiciones para que se cuenten las repeticiones
                
            # Rep counter logic legs
            if angle_r2 > 160 and angle_l2 >160:
                stage_leg ="up"
            if angle_r2 < 110 and angle_l2 <110 and stage_leg=="up":
                stage_leg ="down"
                counter +=1    
                
                
        except:
            pass

#Definimos la caja de texto y los mismos textos que aparecen en pantalla        
        # Setup status box
        cv2.rectangle(image, (0,0), (150,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data left
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage_leg, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255,255,255), 2, cv2.LINE_AA)     
        
        
# Definimos condicion para cambio de color de "esqueleto"                
        if 150 < angle_r < 170 and 150 < angle_l < 170 or stage_leg =="down":
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(35,130,45), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(35,130,45), thickness=2, circle_radius=2) 
                                 )         
        else:
        
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), #punto 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)  #linea
                                 )               
        
        
        
        cv2.imshow('Mediapipe Feed', image)

# Cierre de programa apretando letra "q"
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()